In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
import talib as ta
import random
random.seed(42)


In [3]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()


groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [4]:
tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Stocks_group' in symbol.path and symbol.spread < 15]
tickers

['AAPLm',
 'ABBVm',
 'ABTm',
 'ADBEm',
 'ADPm',
 'AMDm',
 'AMGNm',
 'AMTm',
 'AMZNm',
 'ATVIm',
 'BAm',
 'BABAm',
 'BACm',
 'BIIBm',
 'BMYm',
 'Cm',
 'CHTRm',
 'CMCSAm',
 'CMEm',
 'COSTm',
 'CSCOm',
 'CSXm',
 'CVSm',
 'EAm',
 'EBAYm',
 'EQIXm',
 'Fm',
 'FBm',
 'GILDm',
 'GOOGLm',
 'HDm',
 'IBMm',
 'INTCm',
 'ISRGm',
 'JNJm',
 'JPMm',
 'KOm',
 'LINm',
 'LLYm',
 'LMTm',
 'MAm',
 'MCDm',
 'MDLZm',
 'MMMm',
 'MOm',
 'MRKm',
 'MSm',
 'NFLXm',
 'NKEm',
 'ORCLm',
 'PEPm',
 'PFEm',
 'PGm',
 'PMm',
 'PYPLm',
 'REGNm',
 'SBUXm',
 'Tm',
 'TMOm',
 'TMUSm',
 'TSLAm',
 'UNHm',
 'UPSm',
 'Vm',
 'VZm',
 'WFCm',
 'WMTm',
 'XOMm',
 'AMCm',
 'BBm',
 'BBBYm',
 'BEKEm',
 'BIDUm',
 'BILIm',
 'BRQSm',
 'BYNDm',
 'CANm',
 'EDUm',
 'FTNTm',
 'FUTUm',
 'IQm',
 'JDm',
 'LIm',
 'NIOm',
 'NTESm',
 'PDDm',
 'RLXm',
 'TALm',
 'TIGRm',
 'TMEm',
 'TSMm',
 'VIPSm',
 'XPEVm',
 'YUMCm',
 'ZTOm',
 'METAm']

In [5]:
# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2014, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

intervals = [
    # mt5.TIMEFRAME_M15,
    mt5.TIMEFRAME_H1,
    # mt5.TIMEFRAME_H2,
    # mt5.TIMEFRAME_H4,
]

symbols = get_data(tickers, intervals, date_from, date_to)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
AAPLm
ABBVm
ABTm
ADBEm
ADPm
AMDm
AMGNm
AMTm
AMZNm
ATVIm
se descarto ATVIm
BAm
BABAm
BACm
BIIBm
BMYm
Cm
CHTRm
CMCSAm
CMEm
COSTm
CSCOm
CSXm
CVSm
EAm
EBAYm
EQIXm
Fm
FBm
GILDm
GOOGLm
HDm
IBMm
INTCm
ISRGm
JNJm
JPMm
KOm
LINm
LLYm
LMTm
MAm
MCDm
MDLZm
MMMm
MOm
MRKm
MSm
NFLXm
NKEm
ORCLm
PEPm
PFEm
PGm
PMm
PYPLm
REGNm
SBUXm
Tm
TMOm
TMUSm
TSLAm
UNHm
UPSm
Vm
VZm
WFCm
WMTm
XOMm
AMCm
se descarto AMCm
BBm
BBBYm
se descarto BBBYm
BEKEm
BIDUm
BILIm
BRQSm
se descarto BRQSm
BYNDm
se descarto BYNDm
CANm
se descarto CANm
EDUm
FTNTm
FUTUm
IQm
JDm
LIm
NIOm
NTESm
PDDm
RLXm
TALm
TIGRm
se descarto TIGRm
TMEm
TSMm
VIPSm
XPEVm
YUMCm
ZTOm
METAm


In [6]:

import numpy as np

tickers = np.unique(list(symbols.keys())).tolist()


max_start_date = None
intervals_start_dates = {}

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates



{16385: Timestamp('2021-10-27 17:00:00+0000', tz='UTC')}

In [7]:
from pandas import Timestamp
limit_date_train = Timestamp('2022-10-01 00:00:00', tz='UTC')
limit_date_train

Timestamp('2022-10-01 00:00:00+0000', tz='UTC')

In [8]:
from backbone.utils.general_purpose import calculate_units_size, diff_pips

class BPercentError(Strategy):
    'Esta estrategia anduvo con LMT, NFLX e IBM'
    
    risk= 1
    bbands_timeperiod = 50
    bband_std = 1.5
    sma_period = 200
    b_open_threshold = 0.95
    b_close_threshold = 0.5
    atr_multiplier = 1.5
    pip_value = 0.1

    def init(self):
        
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.upper_band, self.middle_band, self.lower_band = self.I(
            ta.BBANDS, self.data.Close, 
            timeperiod=self.bbands_timeperiod, 
            nbdevup=self.bband_std, 
            nbdevdn=self.bband_std
        )
        
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)

    def next(self):
        actual_close = self.data.Close[-1]
        b_percent = (actual_close - self.lower_band[-1]) / (self.upper_band[-1] - self.lower_band[-1])
        
        if self.position:
            if self.position.is_long:
                if b_percent >= self.b_close_threshold:
                    self.position.close()

            if self.position.is_short:
                if b_percent <= 1 - self.b_close_threshold:
                    self.position.close()

        else:

            if b_percent <= 1 - self.b_open_threshold and actual_close > self.sma[-1]:
                self.buy(
                    size=1,
                )
                
            if b_percent >= self.b_open_threshold and actual_close < self.sma[-1]:
                self.buy(
                    size=1,
                )

In [17]:
from backbone.utils.general_purpose import calculate_units_size, diff_pips

class BPercent(Strategy):    
    risk= 1
    bbands_timeperiod = 50
    bband_std = 1.5
    sma_period = 200
    b_open_threshold = 0.95
    b_close_threshold = 0.5
    atr_multiplier = 1.8
    pip_value = 0.1

    def init(self):
        
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.upper_band, self.middle_band, self.lower_band = self.I(
            ta.BBANDS, self.data.Close, 
            timeperiod=self.bbands_timeperiod, 
            nbdevup=self.bband_std, 
            nbdevdn=self.bband_std
        )
        
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)

    def next(self):
        actual_close = self.data.Close[-1]
        b_percent = (actual_close - self.lower_band[-1]) / (self.upper_band[-1] - self.lower_band[-1])
        
        if self.position:
            if self.position.is_long:
                if b_percent >= self.b_close_threshold:
                    self.position.close()

            if self.position.is_short:
                if b_percent <= 1 - self.b_close_threshold:
                    self.position.close()

        else:
            if b_percent >= self.b_open_threshold and actual_close < self.sma[-1]:

                sl_price = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                units = 1 if units == 0 else units
                
                self.buy(
                    size=units,
                    sl=sl_price
                )
                
            if b_percent <= 1 - self.b_open_threshold and actual_close > self.sma[-1]:
                sl_price = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                units = 1 if units == 0 else units

                self.sell(
                    size=units,
                    sl=sl_price    
                )

In [23]:
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression

strategies = [
    BPercentError
]

experiments = parameter_combinations = list(itertools.product(
    tickers, intervals, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:
    start_date = intervals_start_dates[interval]

    frac_df = symbols[ticker][interval].loc[start_date:limit_date_train]

    bt_train = Backtest(
        frac_df, 
        strategy,
        commission=7e-4,
        cash=1_000, 
        margin=1/30
    )
    
    stats = bt_train.run()
    
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)

    df_stats = pd.DataFrame({
        'strategy':[strategy.__name__],
        'ticker':[ticker],
        'interval':[interval],
        'stability_ratio':[stability_ratio],
        'return':[stats['Return [%]']],
        'final_eq':[stats['Equity Final [$]']],
        'drawdown':[stats['Max. Drawdown [%]']],
        'drawdown_duration':[stats['Max. Drawdown Duration']],
        'win_rate':[stats['Win Rate [%]']], 
        'sharpe_ratio':[stats['Sharpe Ratio']],
        'trades':[stats['# Trades']],
        'avg_trade_percent':[stats['Avg. Trade [%]']],
        'exposure':[stats['Exposure Time [%]']],
        'final_equity':[stats['Equity Final [$]']],
        'Duration':[stats['Duration']],

    })

    performance = pd.concat([performance, df_stats])

performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)

performance.drawdown_duration = pd.to_timedelta(performance.drawdown_duration)

performance.drawdown_duration = performance.drawdown_duration.dt.days

C:\Users\SaidJ\AppData\Local\Temp\ipykernel_18424\1061890029.py:20: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).

C:\Users\SaidJ\AppData\Local\Temp\ipykernel_18424\1061890029.py:20: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).

C:\Users\SaidJ\AppData\Local\Temp\ipykernel_18424\1061890029.py:20: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).

C:\Users\SaidJ\AppData\Local\Temp\ipykernel_18424\1061890029.py:20: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported

In [24]:

filter_performance = performance[
    (performance['return']>0) 
    & (performance['return/dd']>1) 
    # & (performance['trades'] > 20)
    # & (performance['stability_ratio'] > 0.55)
].sort_values(by=['stability_ratio'], ascending=[False]).drop_duplicates(subset=['ticker'], keep='first').head(10)

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BPercentError,EDUm,16385,0.924695,0.505305,1005.053046,0.157174,47,61.290323,1.545851,31,0.961460,8.002481,1005.053046,338 days 02:00:00,3.214941,1.513387
0,BPercentError,TMUSm,16385,0.883121,2.475341,1024.753413,1.171389,63,62.962963,1.452278,27,0.716409,15.190652,1024.753413,338 days 02:00:00,2.113167,3.798648
0,BPercentError,TALm,16385,0.868983,0.216341,1002.163407,0.055948,56,60.526316,1.862258,38,1.287995,14.368932,1002.163407,338 days 02:00:00,3.866840,0.750584
0,BPercentError,LIm,16385,0.846671,0.816986,1008.169858,0.394546,67,55.263158,1.746315,38,0.660013,10.798946,1008.169858,338 days 02:00:00,2.070698,2.146274
0,BPercentError,MOm,16385,0.808321,0.428711,1004.287114,0.087610,150,56.521739,1.886454,23,0.346583,6.334563,1004.287114,338 days 02:00:00,4.893407,1.252717
0,BPercentError,BIDUm,16385,0.802781,2.139159,1021.391589,1.840572,116,41.860465,0.862994,43,0.324220,10.787825,1021.391589,338 days 02:00:00,1.162225,2.849772
0,BPercentError,AAPLm,16385,0.755347,2.416847,1024.168473,0.927186,83,60.655738,1.511841,61,0.256582,15.252047,1024.168473,338 days 02:00:00,2.606648,5.175761
0,BPercentError,BMYm,16385,0.614807,0.362697,1003.626969,0.346876,91,57.142857,0.880963,21,0.267382,13.837638,1003.626969,338 days 02:00:00,1.045609,0.832379
0,BPercentError,ABBVm,16385,0.491010,1.385557,1013.855568,1.036549,91,58.333333,1.213344,36,0.278079,13.284133,1013.855568,338 days 02:00:00,1.336701,2.456671


In [20]:
for ticker, interval in zip(portfolio, intervals):
    frac_df = symbols[ticker][interval].loc[start_date:limit_date_train]
    
    bt_train = Backtest(
        frac_df, 
        BPercent,
        commission=7e-4,
        cash=100_000, 
        margin=1/30
    )

    stats = bt_train.run()
    bt_train.plot(filename=f'./plots/{ticker}.html', resample=False)

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

# WFO

In [21]:
from backtest.utils import plot_full_equity_curve, walk_forward

def optim_func_2(stats):
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)
    
    return (stats['Return [%]'] /  (1 + (-1*stats['Max. Drawdown [%]']))) * np.log(1 + stats['# Trades']) * stability_ratio
    
stats_per_symbol = {}
lookback_bars = 1500
validation_bars = 250
warmup_bars = 200

wfo_performance = pd.DataFrame()
for ticker, interval in zip(portfolio, intervals):
    
    try:
    
        params = {
            'b_open_threshold' : [0.9, 0.95, 0.1, 1.5],
            'b_close_threshold': [0.5, 0.6, 0.8, 0.9],
            'maximize': optim_func_2
        }
        
        fracc_df = symbols[ticker][interval]

        wfo_stats = walk_forward(
            BPercent,
            fracc_df, 
            lookback_bars=lookback_bars,
            validation_bars=validation_bars,
            warmup_bars=warmup_bars, 
            params=params,
            commission=7e-4, 
            margin=1/30, 
            cash=10_000,
            verbose=False
        )
        
        stats_per_symbol[ticker] = {}
        stats_per_symbol[ticker][interval] = wfo_stats
        
        df_equity = wfo_stats['_equity']
        trades = wfo_stats['_trades']
        
        
        plot_full_equity_curve(df_equity, title=f'{ticker}, {interval}')
        
        # Calculo el stability ratio
        x = np.arange(df_equity.shape[0]).reshape(-1, 1)
        reg = LinearRegression().fit(x, df_equity.Equity)
        stability_ratio = reg.score(x, df_equity.Equity)

        largest_lost = min(trades.PnL)
        
        # Extraigo metricas
        df_stats = pd.DataFrame({
            'strategy':[strategy.__name__],
            'ticker':[ticker],
            'interval':[interval],
            'stability_ratio':[stability_ratio],
            'return':[wfo_stats['Return [%]']],
            'largest_lost':[largest_lost],
            'drawdown':[wfo_stats['Max. Drawdown [%]']],
            'final_eq':[wfo_stats['Equity Final [$]']],
            'drawdown_duration':[wfo_stats['Max. Drawdown Duration']],
            'win_rate':[wfo_stats['Win Rate [%]']], 
            'sharpe_ratio':[wfo_stats['Sharpe Ratio']],
            'trades':[wfo_stats['# Trades']],
            'avg_trade_percent':[wfo_stats['Avg. Trade [%]']],
            'exposure':[wfo_stats['Exposure Time [%]']],
            'final_equity':[wfo_stats['Equity Final [$]']],
            'Duration':[wfo_stats['Duration']],

        })

        wfo_performance = pd.concat([wfo_performance, df_stats])
        
    except:
        print(f'no se pudo con {ticker}')

wfo_performance['return/dd'] = wfo_performance['return'] / -wfo_performance['drawdown']
wfo_performance['drawdown'] = -wfo_performance['drawdown']
wfo_performance['custom_metric'] = (wfo_performance['return'] / (1 + wfo_performance.drawdown)) * np.log(1 + wfo_performance.trades)

wfo_performance.drawdown_duration = pd.to_timedelta(wfo_performance.drawdown_duration)
wfo_performance.drawdown_duration = wfo_performance.drawdown_duration.dt.days

wfo_performance.sort_values(by='return/dd', ascending=False)

,strategy,ticker,interval,stability_ratio,return,largest_lost,drawdown,final_eq,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BPercent,VIPSm,16385,0.609492,23.039495,-373.631104,7.975475,12303.949496,301,48.780488,0.804866,246,0.164858,5.002009,12303.949496,1038 days 02:00:00,2.888793,14.142262
0,BPercent,NFLXm,16385,0.184035,3.038174,-114.009541,4.671725,10303.817425,413,42.931937,0.312520,191,0.027643,2.744160,10303.817425,1038 days 02:00:00,0.650332,2.816284
0,BPercent,EDUm,16385,0.638242,6.213924,-304.038300,10.346092,10621.392406,219,47.058824,0.247066,204,-0.014029,5.016116,10621.392406,1038 days 02:00:00,0.600606,2.915258
0,BPercent,CVSm,16385,0.655502,3.172625,-284.435190,5.675561,10317.262528,576,50.442478,0.340344,113,0.035414,2.993170,10317.262528,1038 days 02:00:00,0.558998,2.250924
0,BPercent,NKEm,16385,0.197001,1.345517,-850.041648,17.467125,10134.551737,291,44.866920,0.050326,263,0.004547,5.002009,10134.551737,1038 days 02:00:00,0.077031,0.406264
0,BPercent,CMEm,16385,0.595448,-12.451108,-283.600350,20.653059,8754.889174,413,44.444444,0.000000,171,-0.075673,4.961832,8754.889174,1038 days 02:00:00,-0.602870,-2.959952
0,BPercent,GOOGLm,16385,0.684945,-23.998603,-576.237641,26.870614,7600.139725,345,42.541436,0.000000,181,-0.215842,5.002009,7600.139725,1038 days 02:00:00,-0.893117,-4.481024
0,BPercent,KOm,16385,0.634650,-17.007486,-257.619750,18.298164,8299.251421,683,41.477273,0.000000,176,-0.090033,4.981920,8299.251421,1038 days 02:00:00,-0.929464,-4.561745
0,BPercent,NIOm,16385,0.929120,-33.778327,-252.207572,35.873109,6622.167260,711,39.879154,0.000000,331,-0.333411,3.503588,6622.167260,1038 days 02:00:00,-0.941606,-5.317907
0,BPercent,JPMm,16385,0.836799,-26.151422,-432.786147,26.771592,7384.857818,323,36.184211,0.000000,152,-0.146404,4.981920,7384.857818,1038 days 02:00:00,-0.976835,-4.736967


In [22]:
wfo_performance[
    (wfo_performance['return'] > 0)
    # & (wfo_performance['trades'] > 20)
].sort_values(by='stability_ratio', ascending=False)

,strategy,ticker,interval,stability_ratio,return,largest_lost,drawdown,final_eq,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BPercent,CVSm,16385,0.655502,3.172625,-284.435190,5.675561,10317.262528,576,50.442478,0.340344,113,0.035414,2.993170,10317.262528,1038 days 02:00:00,0.558998,2.250924
0,BPercent,EDUm,16385,0.638242,6.213924,-304.038300,10.346092,10621.392406,219,47.058824,0.247066,204,-0.014029,5.016116,10621.392406,1038 days 02:00:00,0.600606,2.915258
0,BPercent,VIPSm,16385,0.609492,23.039495,-373.631104,7.975475,12303.949496,301,48.780488,0.804866,246,0.164858,5.002009,12303.949496,1038 days 02:00:00,2.888793,14.142262
0,BPercent,NKEm,16385,0.197001,1.345517,-850.041648,17.467125,10134.551737,291,44.866920,0.050326,263,0.004547,5.002009,10134.551737,1038 days 02:00:00,0.077031,0.406264
0,BPercent,NFLXm,16385,0.184035,3.038174,-114.009541,4.671725,10303.817425,413,42.931937,0.312520,191,0.027643,2.744160,10303.817425,1038 days 02:00:00,0.650332,2.816284
